<a href="https://colab.research.google.com/github/Wishva23/GUVI_CODEKATA_PYTHON_CODE/blob/master/API_ASSIGNMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir apiproject
! cd myproject
! python -m venv venv
! . venv/bin/activate
! pip install Flask

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Define some example product data
PRODUCTS = [
    {"name": "Product 1", "barcode": "34567890", "brand": "Brand 1", "description": "This is sample description", "price": 200, "available": True},
    {"name": "Product 2", "barcode": "4567890", "brand": "Brand 2", "description": "This is sample description", "price": 100, "available": False},
    {"name": "Product 3", "barcode": "987654", "brand": "Brand 3", "description": "This is sample description", "price": 150, "available": True},
    {"name": "Product 4", "barcode": "12345678", "brand": "Brand 4", "description": "This is sample description", "price": 250, "available": False},
    {"name": "Product 5", "barcode": "23456765", "brand": "Brand 5", "description": "This is sample description", "price": 300, "available": True},
    {"name": "Product 6", "barcode": "23423452", "brand": "Brand 6", "description": "This is sample description", "price": 350, "available": True},
    {"name": "Product 7", "barcode": "987123", "brand": "Brand 7", "description": "This is sample description", "price": 180, "available": True},
    {"name": "Product 8", "barcode": "1256789", "brand": "Brand 8", "description": "This is sample description", "price": 120, "available": False},
    {"name": "Product 9", "barcode": "34562718", "brand": "Brand 9", "description": "This is sample description", "price": 110, "available": True},
    {"name": "Product 10", "barcode": "3.46E+08", "brand": "Brand 10", "description": "This is sample description", "price": 90, "available": False},
    {"name": "Product 11", "barcode": "667788", "brand": "Brand 10", "description": "This is sample description", "price": 90, "available": False},
    {"name": "Product 12", "barcode": "887799", "brand": "Brand 10", "description": "This is sample description", "price": 90, "available": False},
    {"name": "Product 13", "barcode": "24342", "brand": "Brand 10", "description": "This is sample description", "price": 90, "available": False},
    {"name": "Product 14", "barcode": "111", "brand": "Brand 10", "description": "This is sample description", "price": 90, "available": False},
    {"name": "Product 15", "barcode": "1015", "brand": "Brand 10", "description": "This is sample description", "price": 90, "available": False},
]

# Define a simple authentication function for demonstration purposes
def authenticate(username, password):
    if username == "admin" and password == "password":
        return True
    else:
        return False


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# A list to store registered users
users = []

# A dictionary to store product data
products = {}

# Registration API
@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    users.append(data)
    return jsonify({'message': 'User registered successfully.'})

# Login API
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    for user in users:
        if user['username'] == data['username'] and user['password'] == data['password']:
            return jsonify({'message': 'Login successful.'})
    return jsonify({'message': 'Invalid username or password.'}), 401

# Product Upload API
@app.route('/upload', methods=['POST'])
def upload():
    file = request.files['file']
    filename = secure_filename(file.filename)
    file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    with open(filename, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            products[row['barcode']] = {
                'name': row['name'],
                'brand': row['brand'],
                'description': row['description'],
                'price': row['price'],
                'available': row['available']
            }
    return jsonify({'message': 'CSV file uploaded successfully.'})

# Product Review API
@app.route('/review', methods=['POST'])
def review():
    data = request.get_json()
    barcode = data['barcode']
    if barcode not in products:
        return jsonify({'message': 'Product not found.'}), 404
    products[barcode]['rating'] = data['rating']
    return jsonify({'message': 'Product review added successfully.'})

# Product view Pagination API
@app.route('/products', methods=['GET'])
def products():
    sort_by = request.args.get('sort_by', 'barcode')
    sort_order = request.args.get('sort_order', 'asc')
    page = request.args.get('page', 1, type=int)
    per_page = request.args.get('per_page', 10, type=int)
    start_index = (page - 1) * per_page
    end_index = start_index + per_page
    sorted_products = sorted(products.values(), key=lambda p: p[sort_by], reverse=sort_order=='desc')
    paginated_products = sorted_products[start_index:end_index]
    return jsonify({'products': paginated_products})

if __name__ == '__main__':
    app.run(debug=True)
